In [1]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

2019-04-26 17:04:12.739000


In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt

In [4]:
from __future__ import division
import pandas as pd ; pd.set_option("display.max_columns",200)
import numpy as np
import sys
import pickle

In [5]:
import sys
sys.path.append(".\Income_tax_function")
import IRPP_from_scratch as ir

In [6]:
import sys
sys.path.append(".\Income_tax_function")
import IRPP_from_scratch as ir

In [7]:
%pwd

u'C:\\Users\\IMPTEMP_A_PACIFIC\\Desktop\\Cohabitant_project(EDP_2015)\\Programme'

In [8]:
store_path = '../Data/hdf/edp_2015_final.h5'
Store = pd.HDFStore(store_path)

In [9]:
Store

<class 'pandas.io.pytables.HDFStore'>
File path: ../Data/hdf/edp_2015_final.h5
/DECES                         frame        (shape->[358338,19])  
/DESCENDANCE                   frame        (shape->[1320391,41]) 
/EAR2004_FAMILLE               frame        (shape->[48316,13])   
/EAR2004_INDIVIDU              frame        (shape->[277933,86])  
/EAR2004_LOGEMENT              frame        (shape->[89892,52])   
/EAR2005_FAMILLE               frame        (shape->[78041,13])   
/EAR2005_INDIVIDU              frame        (shape->[283781,86])  
/EAR2005_LOGEMENT              frame        (shape->[91733,52])   
/EAR2006_FAMILLE               frame        (shape->[78227,13])   
/EAR2006_INDIVIDU              frame        (shape->[287786,86])  
/EAR2006_LOGEMENT              frame        (shape->[92761,52])   
/EAR2007_FAMILLE               frame        (shape->[78230,13])   
/EAR2007_INDIVIDU              frame        (shape->[283228,86])  
/EAR2007_LOGEMENT              frame        (shape

In [10]:
def load_fisc_i_by_year(year = 2014):
    df_fisci = pd.read_hdf(store_path, 'FISC_INDIVIDU_{}'.format(year))
    return df_fisci
def load_fisc_r_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_REVENU_{}'.format(year))
    return df_fiscr
def load_fisc_l_by_year(year = 2014):
    df_fiscr = pd.read_hdf(store_path, 'FISC_LOGEMENT_{}'.format(year))
    return df_fiscr

def load_df_fiscrevdet_by_year(year = 2014):
    df_fiscrevdet = pd.read_hdf(store_path, 'FISC_REVDET_{}'.format(year))
    return df_fiscrevdet
    

### ID duplicates (drop)

In [11]:
pickle_path = u"./Pickle/"

In [12]:
final_select = "coucou"

In [13]:
%%time

for year in range(2010,2015):
    print(year)
    df_fisci = load_fisc_i_by_year(year)
    print year
    assert df_fisci.AN_FISC.iloc[0] == year+1
    id_with_one_duplicated_individual_drop = df_fisci[df_fisci[['ID_FISC_FOY_DIFF', 'TYPE_FISC', 'ANAIS']].duplicated()].ID_FISC_LOG_DIFF
    #Two fiscal units
    grpby_fisci = df_fisci.groupby('ID_FISC_LOG_DIFF')
    grpby_fisci_nunique_id = grpby_fisci.ID_FISC_FOY_DIFF.nunique()
    two_fiscal_unit_id = grpby_fisci_nunique_id[(grpby_fisci_nunique_id ==2)].index
    #Id conjoint declarant( drop)
    typ_fisc_equal_two_id_drop = df_fisci[df_fisci.TYPE_FISC == '2'].ID_FISC_LOG_DIFF
    #Id avec majeur ratachés (drop)
    id_majeur_rataches_drop = df_fisci[df_fisci.TYPE_FISC.isin(['3','4','5','6','7', '8'])].ID_FISC_LOG_DIFF
    #ID menage pas en couple avec enfants ou complexe
    df_fiscr = load_fisc_r_by_year(year)
    assert df_fiscr.AN_FISC.iloc[0] == year+1
    id_couple_avec_enfants = df_fiscr[df_fiscr.TYPMEN9.isin([41,42,43])].ID_FISC_LOG_DIFF


    to_select_id_drop = set()
    to_select_id_drop = to_select_id_drop.union(id_with_one_duplicated_individual_drop )
    to_select_id_drop =  to_select_id_drop.union(typ_fisc_equal_two_id_drop)
    to_select_id_drop =  to_select_id_drop.union(id_majeur_rataches_drop )
    to_select_id_keep = set(two_fiscal_unit_id)
    to_select_id_keep = to_select_id_keep.intersection(id_couple_avec_enfants  )
    final_select = to_select_id_keep - to_select_id_drop #set difference

#     pickle.dump(final_select, 
#                 open(pickle_path +
#                     "specific_select/one_duplicated_individual_drop_rev_{}.p".format(year), 'wb'))

#     pickle.dump(final_select, 
#                 open(pickle_path +
#                     "specific_select/two_fiscal_unit_id_rev_{}.p".format(year), 'wb'))
#     pickle.dump(final_select, 
#                 open(pickle_path +
#                     "specific_select/typ_fisc_equal_two_id_drop_rev{}.p".format(year), 'wb'))
#     pickle.dump(final_select, 
#                 open(pickle_path +
#                     "specific_select/id_majeur_rataches_drop_rev_{}.p".format(year), 'wb'))

#     pickle.dump(final_select, 
#                 open(pickle_path +
#                     "specific_select/id_couple_avec_enfants_rev_{}.p".format(year), 'wb'))






    pickle.dump(final_select, 
                open(pickle_path + "final_select_rev_{}.p".format(year), 'wb'))
    
    del df_fiscr

2010
2010
2011
2011
2012
2012
2013
2013
2014
2014
Wall time: 30min 7s


In [14]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)
stop_time = start_time
#clear  memory
%reset -f

('now: ', datetime.datetime(2019, 4, 26, 17, 34, 23, 714000))
('execution_time: ', datetime.timedelta(0, 1810, 975000))
